In [2]:
import heapq
from collections import defaultdict

graph = {
    "S": {"A": 5, "B": 9, "C": 6, "D": 6},
    "A": {"B": 3, "G1": 9},
    "B": {"A": 2, "C": 1},
    "C": {"D": 2, "F": 7, "G2": 5},
    "D": {"E": 2},
    "E": {"G3": 7},
    "F": {"G3": 8, "D": 2},
    "G1": {},
    "G2": {},
    "G3": {},
}

def ucs_verbose(start, goal, graph):
    # best_cost keeps the best g(n) seen for each node
    best_cost = defaultdict(lambda: float("inf"))
    best_cost[start] = 0

    # priority queue items: (cost, tiebreaker, node, path_list)
    # tiebreaker ensures deterministic behavior when costs tie
    counter = 0
    pq = []
    heapq.heappush(pq, (0, counter, start, [start]))
    counter += 1

    explored = set()
    step = 0

    def frontier_snapshot(pq):
        # Show as list of (node, g) sorted by (g, node) for readability
        snap = sorted([(item[2], item[0]) for item in pq], key=lambda x:(x[1], x[0]))
        return "[" + ", ".join(f"({n},{g})" for n, g in snap) + "]"

    print("=== Uniform-Cost Search (verbose) ===")
    print(f"Start: {start}, Goal: {goal}\n")

    while pq:
        g, _, node, path = heapq.heappop(pq)
        step += 1
        print(f"Step {step}) Pop {node} with cost {g}; path = {' → '.join(path)}")

        # If we pop the goal, UCS guarantees optimality
        if node == goal:
            print("\n>>> Goal reached.")
            print(f"Optimal path: {' → '.join(path)}")
            print(f"Total cost: {g}")
            return path, g

        if node in explored:
            print("  (Already explored with a better cost; skip)")
            print(f"  Frontier: {frontier_snapshot(pq)}\n")
            continue

        explored.add(node)

        # Expand neighbors (alphabetical to break ties consistently)
        for nbr in sorted(graph[node].keys()):
            w = graph[node][nbr]
            new_cost = g + w
            if new_cost < best_cost[nbr]:
                best_cost[nbr] = new_cost
                heapq.heappush(pq, (new_cost, counter, nbr, path + [nbr]))
                counter += 1
                print(f"  Relax: {node} → {nbr} (edge {w}) → new g({nbr}) = {new_cost}")
            else:
                print(f"  Skip:  {node} → {nbr} (edge {w}) → candidate {new_cost} not better than {best_cost[nbr]}")

        print(f"  Frontier: {frontier_snapshot(pq)}\n")

    print("No path found.")
    return None, float("inf")


